In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
import itertools
import random

In [4]:
# adjust for firing freq
slope_f = 10 #2
mn_b = 20 #60
mn_a = 0

cm = 10
g_L = 2.5
E_L = -61
V_T = -50
V_thr = 0
V_reset = -58
mn_tau_w = 30

time = 5000
delta_t = 0.1
n_timesteps = len(np.arange(0,time,delta_t))
n_ains = 1

In [ ]:
ain_w = np.zeros((n_ains,1))
ain_voltage_vec = np.zeros((n_ains,1))
ain_voltage_vec[:,:] += E_L
ain_voltage_record = np.zeros((1,n_timesteps))
ain_spike_times = [[] for m in range(n_ains)]
vs_to_plot = []
ws = []
spike_count = 0

# want aIN to fire at 40Hz with 50pA of current

for idx,t in enumerate(np.arange(0,time,delta_t)):
    I = 100
    for ain in range(n_ains):
        if ain_voltage_vec[ain,-1] >= V_thr:
            ain_voltage_vec[ain,-1] = V_reset
    if ain_voltage_vec[0,-1] < -45 and ain_voltage_vec[0,-1] > -70:       
        vs_to_plot.append(ain_voltage_vec[0,-1])
        ws.append(ain_w[0,-1])
    ain_voltage_vec[:,-1] += delta_t*((-g_L*(ain_voltage_vec[:,-1] - E_L) + g_L * slope_f * np.exp((ain_voltage_vec[:,-1] - V_T)/slope_f) \
                                       + I - ain_w[:,-1]) / cm)
    ain_w += delta_t*((mn_a * (ain_voltage_vec - E_L) - ain_w) / mn_tau_w)
    for ain in range(n_ains):
        if ain_voltage_vec[ain,-1] >= V_thr:
            ain_voltage_vec[ain,-1] = V_thr
            ain_spike_times[ain].append(t)
            spike_count += 1
            ain_w[ain,-1] += mn_b
    ain_voltage_record[:,idx] += ain_voltage_vec[:,:].squeeze()
print(spike_count/5)
x = np.arange(0,time,delta_t) 

vnull_ws = []
wnull_ws = []
vnull = np.arange(-70,-45,1)
for V in np.arange(-70,-45,1):
    vnull_w = -g_L * (V - E_L) + g_L * slope_f * np.exp((V - V_T)/slope_f) + I
    wnull_w = mn_a*(V - E_L)
    vnull_ws.append(vnull_w)
    wnull_ws.append(wnull_w)
plt.figure(figsize=(20, 5))
plt.plot(x, ain_voltage_record[-1,:])
plt.title('Voltage record',fontsize='16')
plt.xlabel('Time (ms)',fontsize='14')
plt.ylabel('Voltage (mV)',fontsize='14')
plt.legend()

In [ ]:
# phase plane analysis

idxs = [0]

for idx,v in enumerate(vs_to_plot):
    if v == V_reset: #and vs_to_plot[idx-1] > -46:
        idxs.append(idx)
split_vs = []
split_ws = []
for idx,val in enumerate(idxs):
    if idx < len(idxs)-1:
        split_vs.append(vs_to_plot[val:idxs[idx+1]])
        split_ws.append(ws[val:idxs[idx+1]])
    else:
        split_vs.append(vs_to_plot[val:])
        split_ws.append(ws[val:])
        

for idx,(traj_v,traj_w) in enumerate(zip(split_vs,split_ws)):
    if idx == 0:
        plt.plot(traj_v,traj_w,color='b',label='Trajectory')
    else:
        plt.plot(traj_v,traj_w,color='b')
plt.plot(vnull, vnull_ws,color='r',label='V nullcine')
plt.plot(vnull, wnull_ws,color='g',label='w nullcine')
plt.title('Phase Plane Representation of Trajectories \n in an AdEx Transient Neuron', fontsize='16')
plt.xlabel('V (mV)',fontsize='14')
plt.ylabel('w (pA)',fontsize='14')
plt.legend()
plt.savefig('phaseplanetransientadex')

In [ ]:
Is = [-50, -25, 0, 25]
# Is = [50, 75, 100, 125]
ain_av_vs = []
ain_fr = []

In [ ]:
for I in Is:
    ain_g_ex = np.zeros((n_ains,1))
    ain_g_inh = np.zeros((n_ains,1))
    ain_w = np.zeros((n_ains,1))
    ain_voltage_vec = np.zeros((n_ains,1))
    ain_voltage_vec[:,:] += E_L
    ain_voltage_record = np.zeros((1,n_timesteps))
    ain_spike_times = [[] for m in range(n_ains)]
    for idx,t in enumerate(np.arange(0,time,delta_t)):
        for ain in range(n_ains):
            if ain_voltage_vec[ain,-1] >= V_thr:
                ain_voltage_vec[ain,-1] = V_reset
        ain_voltage_vec[:,-1] += delta_t*((-g_L*(ain_voltage_vec[:,-1] - E_L) + g_L * slope_f * np.exp((ain_voltage_vec[:,-1] - V_T)/slope_f) \
                                            - ain_g_ex[:,-1] * (ain_voltage_vec[:,-1] - E_ex) \
                                            - ain_g_inh[:,-1] * (ain_voltage_vec[:,-1] - E_inh) + I - ain_w[:,-1]) / cm)
        ain_w += delta_t*((mn_a * (ain_voltage_vec - E_L) - ain_w) / mn_tau_w)
        ain_g_ex += delta_t*(-ain_g_ex/tau_ex)
        ain_g_inh += delta_t*(-ain_g_inh/tau_inh)
        for ain in range(n_ains):
            if ain_voltage_vec[ain,-1] >= V_thr:
                ain_voltage_vec[ain,-1] = V_thr
                ain_spike_times[ain].append(t)
                ain_w[ain,-1] += mn_b
        ain_voltage_record[:,idx] += ain_voltage_vec[:,:].squeeze()
    ain_av_v = ain_voltage_record.mean()
    ain_av_vs.append(ain_av_v)
    ain_fr.append(len(ain_spike_times)/5)

x = np.arange(0,time,delta_t) 

plt.plot(Is, ain_av_vs)
plt.title('Current vs Voltage',fontsize='16')
plt.xlabel('Current (pA)',fontsize='14')
plt.ylabel('Voltage (mV)',fontsize='14')
plt.legend()
# plt.savefig('adexcpg')